# GPU Kernels with PyTorch Executions and Debugging

In [ ]:
!nvidia-smi


Mon Apr  7 16:05:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%bash
# Write the CUDA program to a file
cat > hello_gpu_print.cu << 'EOF'
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>

// Define a simple struct to hold thread information.
struct ThreadInfo {
    int global_id;
    int block;   // Block index
    int thread;  // Thread index within the block
};

// Kernel that writes each thread's info into an array.
__global__ void helloGPU(ThreadInfo* out, int total_threads) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if (tid < total_threads) {
        out[tid].block = blockIdx.x;
        out[tid].thread = threadIdx.x;
        out[tid].global_id = tid;
    }
}

int main() {
    int blocks = 2;
    int threadsPerBlock = 5;
    int total_threads = blocks * threadsPerBlock;

    // Allocate device memory for the output array.
    ThreadInfo* d_out;
    cudaMalloc((void**)&d_out, total_threads * sizeof(ThreadInfo));

    // Launch the kernel.
    helloGPU<<<blocks, threadsPerBlock>>>(d_out, total_threads);
    // Wait for the GPU to finish execution.
    cudaDeviceSynchronize();

    // Allocate host memory and copy the device data to the host.
    ThreadInfo* h_out = (ThreadInfo*)malloc(total_threads * sizeof(ThreadInfo));
    cudaMemcpy(h_out, d_out, total_threads * sizeof(ThreadInfo), cudaMemcpyDeviceToHost);

    // Print the messages from the host.
    for (int i = 0; i < total_threads; i++) {
        printf("Hello GPU from thread %d in block %d with global id %d\n", h_out[i].thread, h_out[i].block, h_out[i].global_id);
    }

    // Free host and device memory.
    free(h_out);
    cudaFree(d_out);

    return 0;
}
EOF

# Compile and run the CUDA program. sm_70 architecture needed for printf
nvcc -arch=sm_70 hello_gpu_print.cu -o hello_gpu_print && ./hello_gpu_print



Hello GPU from thread 0 in block 0 with global id 0
Hello GPU from thread 1 in block 0 with global id 1
Hello GPU from thread 2 in block 0 with global id 2
Hello GPU from thread 3 in block 0 with global id 3
Hello GPU from thread 4 in block 0 with global id 4
Hello GPU from thread 0 in block 1 with global id 5
Hello GPU from thread 1 in block 1 with global id 6
Hello GPU from thread 2 in block 1 with global id 7
Hello GPU from thread 3 in block 1 with global id 8
Hello GPU from thread 4 in block 1 with global id 9


In [ ]:
%%bash
cat > matmul.cu << 'EOF'
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <chrono>

// Macro to convert 2D indices into a 1D index.
#define IDX(row, col, n) ((row)*(n) + (col))

__global__ void matrixMulKernel(float* A, float* B, float* C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Ensure that we are within bounds in case n is not a multiple of the block size.
    if (row < n && col < n) {
        float sum = 0.0f;
        for (int k = 0; k < n; k++) {
            sum += A[IDX(row, k, n)] * B[IDX(k, col, n)];
        }
        C[IDX(row, col, n)] = sum;
    }
}

void matrixMulCPU(const float* A, const float* B, float* C, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            float sum = 0.0f;
            for (int k = 0; k < n; k++) {
                sum += A[i * n + k] * B[k * n + j];
            }
            C[i * n + j] = sum;
        }
    }
}

bool compareMatrices(const float* ref, const float* data, int n, float tol = 1e-3f) {
    for (int i = 0; i < n * n; i++) {
        if (fabs(ref[i] - data[i]) > tol) {
            return false;
        }
    }
    return true;
}

int main() {
    int n = 512;
    size_t bytes = n * n * sizeof(float);

    float *h_A = (float*)malloc(bytes);
    float *h_B = (float*)malloc(bytes);
    float *h_C = (float*)malloc(bytes);
    float *h_C_cpu = (float*)malloc(bytes);
    for (int i = 0; i < n * n; i++) {
        h_A[i] = (float)rand() / RAND_MAX;
        h_B[i] = (float)rand() / RAND_MAX;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, bytes);
    cudaMalloc((void**)&d_B, bytes);
    cudaMalloc((void**)&d_C, bytes);

    cudaMemcpy(d_A, h_A, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, bytes, cudaMemcpyHostToDevice);

    dim3 blockDim(16, 16);
    dim3 gridDim((n + blockDim.x - 1) / blockDim.x,
                 (n + blockDim.y - 1) / blockDim.y);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    matrixMulKernel<<<gridDim, blockDim>>>(d_A, d_B, d_C, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA kernel launch error: %s\n", cudaGetErrorString(err));
        return -1;
    }


    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("GPU kernel execution time: %f ms\n", elapsedTime);

    cudaMemcpy(h_C, d_C, bytes, cudaMemcpyDeviceToHost);

    auto cpu_start = std::chrono::high_resolution_clock::now();
    matrixMulCPU(h_A, h_B, h_C_cpu, n);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> cpu_duration = cpu_end - cpu_start;
    printf("CPU computation time: %f ms\n", cpu_duration.count());

    if (compareMatrices(h_C_cpu, h_C, n)) {
        printf("Results match!\n");
    } else {
        printf("Results differ!\n");
    }

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_C_cpu);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}
EOF

# Compile the CUDA program.
nvcc -arch=sm_70 matmul.cu -o matmul && ./matmul


GPU kernel execution time: 1.300832 ms
CPU computation time: 804.479288 ms
Results match!


In [ ]:
%%bash
cat > matmul.cu << 'EOF'
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <chrono>

// Macro to convert 2D indices into a 1D index.
#define IDX(row, col, n) ((row)*(n) + (col))

// Define TILE_WIDTH as a constant. Here we choose 16.
#define TILE_WIDTH 16

// GPU Kernel: Tiled Matrix Multiplication using Shared Memory.
__global__ void matrixMulShared(float* A, float* B, float* C, int n) {
    // 1. Compute the global row and column index for each thread.
    int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int col = blockIdx.x * TILE_WIDTH + threadIdx.x;

    // 2. Declare shared memory arrays to hold a tile (sub-matrix) from A and from B.
    __shared__ float A_tile[TILE_WIDTH][TILE_WIDTH];
    __shared__ float B_tile[TILE_WIDTH][TILE_WIDTH];

    float sum = 0.0f;  // This will accumulate our partial sums.

    // 3. Loop over the tiles. The loop runs for ceil(n / TILE_WIDTH) iterations.
    for (int t = 0; t < (n + TILE_WIDTH - 1) / TILE_WIDTH; t++) {
        // --- Load data into shared memory tiles from global memory ---

        // For matrix A:
        //    The element of A to be loaded is at row 'row' and column 't * TILE_WIDTH + threadIdx.x'.
        //    We must check that these indices are within the bounds of the matrix.
        if (row < n && t * TILE_WIDTH + threadIdx.x < n)
            A_tile[threadIdx.y][threadIdx.x] = A[row * n + t * TILE_WIDTH + threadIdx.x];
        else
            A_tile[threadIdx.y][threadIdx.x] = 0.0f;

        // For matrix B:
        //    The element of B to be loaded is at row 't * TILE_WIDTH + threadIdx.y' and column 'col'.
        //    Again, boundary checking is necessary.
        if (col < n && t * TILE_WIDTH + threadIdx.y < n)
            B_tile[threadIdx.y][threadIdx.x] = B[(t * TILE_WIDTH + threadIdx.y) * n + col];
        else
            B_tile[threadIdx.y][threadIdx.x] = 0.0f;

        // Synchronize threads to ensure the tile is completely loaded.
        __syncthreads();

        // --- Compute partial sum for the current tile ---
        // Each thread calculates the dot product for one output element.
        for (int i = 0; i < TILE_WIDTH; i++) {
            sum += A_tile[threadIdx.y][i] * B_tile[i][threadIdx.x];
        }

        // Synchronize threads before loading the next tile.
        __syncthreads();
    }

    // 4. Write the final result to the global result matrix C.
    if (row < n && col < n)
        C[row * n + col] = sum;
}

// CPU Version of Matrix Multiplication (for verification and timing).
void matrixMulCPU(const float* A, const float* B, float* C, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            float sum = 0.0f;
            for (int k = 0; k < n; k++) {
                sum += A[i * n + k] * B[k * n + j];
            }
            C[i * n + j] = sum;
        }
    }
}

// Compare two matrices element-wise within a tolerance.
bool compareMatrices(const float* ref, const float* data, int n, float tol = 1e-3f) {
    for (int i = 0; i < n * n; i++) {
        if (fabs(ref[i] - data[i]) > tol) {
            return false;
        }
    }
    return true;
}

int main() {
    int n = 512;  // Dimension of the matrices (n x n)
    size_t bytes = n * n * sizeof(float);

    // Allocate host memory for matrices A, B, and result matrices (for both GPU and CPU versions).
    float *h_A = (float*)malloc(bytes);
    float *h_B = (float*)malloc(bytes);
    float *h_C = (float*)malloc(bytes);     // To store GPU result.
    float *h_C_cpu = (float*)malloc(bytes);   // To store CPU result.

    // Initialize matrices A and B with random values.
    for (int i = 0; i < n * n; i++) {
        h_A[i] = (float)rand() / RAND_MAX;
        h_B[i] = (float)rand() / RAND_MAX;
    }

    // Allocate device memory.
    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, bytes);
    cudaMalloc((void**)&d_B, bytes);
    cudaMalloc((void**)&d_C, bytes);

    // Copy matrices A and B from host to device.
    cudaMemcpy(d_A, h_A, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, bytes, cudaMemcpyHostToDevice);

    // Define block and grid dimensions.
    // We use a block of size TILE_WIDTH x TILE_WIDTH.
    dim3 blockDim(TILE_WIDTH, TILE_WIDTH);
    // The grid dimensions are computed to cover an n x n matrix.
    dim3 gridDim((n + blockDim.x - 1) / blockDim.x,
                 (n + blockDim.y - 1) / blockDim.y);

    // Create CUDA events for timing the GPU kernel.
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Launch the shared memory matrix multiplication kernel.
    matrixMulShared<<<gridDim, blockDim>>>(d_A, d_B, d_C, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Check for any kernel launch errors.
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA kernel launch error: %s\n", cudaGetErrorString(err));
        return -1;
    }

    // Measure GPU execution time.
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("GPU kernel execution time: %f ms\n", elapsedTime);

    // Copy the result matrix from device to host.
    cudaMemcpy(h_C, d_C, bytes, cudaMemcpyDeviceToHost);

    // Perform matrix multiplication on the CPU for verification and timing.
    auto cpu_start = std::chrono::high_resolution_clock::now();
    matrixMulCPU(h_A, h_B, h_C_cpu, n);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> cpu_duration = cpu_end - cpu_start;
    printf("CPU computation time: %f ms\n", cpu_duration.count());

    // Verify that the GPU and CPU results match.
    if (compareMatrices(h_C_cpu, h_C, n)) {
        printf("Results match!\n");
    } else {
        printf("Results differ!\n");
    }

    // Free device and host memory.
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_C_cpu);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}
EOF

# Compile the CUDA program with an appropriate architecture flag and run.
nvcc -arch=sm_70 matmul.cu -o matmul && ./matmul


GPU kernel execution time: 0.894240 ms
CPU computation time: 1347.606422 ms
Results match!


Learned memory coalescing and banking conflicts. Implemeneted and tested pytorch profiling

# Avoiding banking conflicts in shared memory access

In [ ]:
#CELL NOT MEANT TO BE RUN (ONLY FOR LEARNING)

Avoiding Bank Conflicts by Padding
One common way to avoid bank conflicts is to add padding to your shared memory arrays so that the stride is not a multiple of the number of banks.

Code Example: Bank Conflict-Free Kernel
Imagine now you have a 2D shared memory array that represents a tile, and you add an extra column as padding.

cpp
Copy
// Define TILE_WIDTH (assume 32 for this example)
#define TILE_WIDTH 32

__global__ void bankConflictFreeKernel(float *input, float *output) {
    // Declare a 2D shared memory array with padding.
    // Note the dimension is [TILE_WIDTH][TILE_WIDTH+1]
    // The extra column helps ensure that consecutive rows start at different banks.
    __shared__ float sdata[TILE_WIDTH][TILE_WIDTH + 1];

    int tid = threadIdx.x; // Overall thread index within a block.
    int row = threadIdx.y;
    int col = threadIdx.x; // For simplicity, assume a 2D block of size TILE_WIDTH x TILE_WIDTH.

    // Each thread loads its element.
    // Because of the padding in the second dimension, the address of each element is:
    //   Address = row * (TILE_WIDTH + 1) + col
    // Now the bank for a given element becomes:
    //   (row*(TILE_WIDTH+1) + col) mod number_of_banks.
    // With TILE_WIDTH = 32 and an extra 1, the stride is 33 instead of 32.
    // This typically avoids the scenario where all accesses fall into the same bank.
    if (row < TILE_WIDTH && col < TILE_WIDTH)
        sdata[row][col] = input[row * TILE_WIDTH + col];

    __syncthreads();

    // For this example, simply write back to global memory.
    // Combine row and col to index the 1D global output.
    if (row < TILE_WIDTH && col < TILE_WIDTH)
        output[row * TILE_WIDTH + col] = sdata[row][col];
}
Explanation:

Here we use a 2D shared array sdata[TILE_WIDTH][TILE_WIDTH+1].

The extra column (making the stride 33 instead of 32) changes the mapping:

For row = 0, elements are at addresses 0, 1, 2, …, 31

For row = 1, elements are at addresses 33, 34, …, 33+31

This alteration means that the starting address of each row is offset by 33 mod 32
=
1
=1 relative to the previous row. Consequently, threads in consecutive rows do not access the same bank, thus minimizing potential bank conflicts.

# Matrix Multiplcation with Shared Memory

In [ ]:
%%bash
cat > matmul_extension.cpp << 'EOF'
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define TILE_WIDTH 16
#define IDX(row, col, n) ((row)*(n) + (col))

// Our shared memory matrix multiplication kernel.
__global__ void matrixMulShared_kernel(const float* A, const float* B, float* C, int n) {
    int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int col = blockIdx.x * TILE_WIDTH + threadIdx.x;

    __shared__ float A_tile[TILE_WIDTH][TILE_WIDTH];
    __shared__ float B_tile[TILE_WIDTH][TILE_WIDTH];

    float sum = 0.0f;
    for (int t = 0; t < (n + TILE_WIDTH - 1) / TILE_WIDTH; t++) {
        if (row < n && t * TILE_WIDTH + threadIdx.x < n)
            A_tile[threadIdx.y][threadIdx.x] = A[row * n + t * TILE_WIDTH + threadIdx.x];
        else
            A_tile[threadIdx.y][threadIdx.x] = 0.0f;

        if (col < n && t * TILE_WIDTH + threadIdx.y < n)
            B_tile[threadIdx.y][threadIdx.x] = B[(t * TILE_WIDTH + threadIdx.y) * n + col];
        else
            B_tile[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();
        for (int i = 0; i < TILE_WIDTH; i++) {
            sum += A_tile[threadIdx.y][i] * B_tile[i][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < n && col < n)
        C[row * n + col] = sum;
}

// Launcher function that wraps kernel launch.
void matrixMulShared_launcher(const torch::Tensor A, const torch::Tensor B, torch::Tensor C, int n) {
    const int threads = TILE_WIDTH;
    dim3 block(threads, threads);
    dim3 grid((n + threads - 1) / threads, (n + threads - 1) / threads);
    matrixMulShared_kernel<<<grid, block>>>(A.data_ptr<float>(), B.data_ptr<float>(), C.data_ptr<float>(), n);
}

// Bind the function using pybind11.
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("matrix_mul_shared", &matrixMulShared_launcher, "Tiled Matrix Multiplication using Shared Memory (CUDA)");
}
EOF


# Custom Activation Function (ClippedReLu) implementation & PyTorch export and comparison

In [ ]:
%%bash
cat > custom_activation.cu <<'EOF'
#include <cuda_runtime.h>
#include <cstdio>

// =====================
// device kernels
// =====================
__global__ void clipped_relu_forward_kernel(const float* x,
                                            float* y,
                                            float cap,
                                            int64_t n)
{
    int64_t i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        float v = x[i];
        v = v < 0.f ? 0.f : v;
        v = v > cap ? cap : v;
        y[i] = v;
    }
}

__global__ void clipped_relu_backward_kernel(const float* x,
                                             const float* gy,
                                             float* gx,
                                             float cap,
                                             int64_t n)
{
    int64_t i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        float g = (x[i] < 0.f || x[i] > cap) ? 0.f : 1.f;
        gx[i] = g * gy[i];
    }
}

extern "C" void clipped_relu_forward_launcher(const float* x,
                                              float* y,
                                              float cap,
                                              int64_t n,
                                              cudaStream_t stream)
{
    const int TPB = 256;
    int blocks = (n + TPB - 1) / TPB;
    clipped_relu_forward_kernel<<<blocks, TPB, 0, stream>>>(x, y, cap, n);
}

extern "C" void clipped_relu_backward_launcher(const float* x,
                                               const float* gy,
                                               float* gx,
                                               float cap,
                                               int64_t n,
                                               cudaStream_t stream)
{
    const int TPB = 256;
    int blocks = (n + TPB - 1) / TPB;
    clipped_relu_backward_kernel<<<blocks, TPB, 0, stream>>>(x, gy, gx, cap, n);
}
EOF


In [ ]:
!nvcc -arch=sm_75 -O3 -shared -Xcompiler -fPIC \
      custom_activation.cu -o custom_activation.so


In [ ]:
import ctypes, torch, math, os

# 3.1 load the .so
lib = ctypes.CDLL(os.path.abspath("custom_activation.so"))

# 3.2 declare argtypes
ptr = ctypes.c_void_p
c_float = ctypes.c_float
c_int64 = ctypes.c_longlong
c_stream = ctypes.c_void_p   # cudaStream_t

lib.clipped_relu_forward_launcher.argtypes  = [ptr, ptr, c_float, c_int64, c_stream]
lib.clipped_relu_backward_launcher.argtypes = [ptr, ptr, ptr, c_float, c_int64, c_stream]
# returns are void ⇒ no restype needed

# 3.3 autograd Function
class ClippedReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, cap: float = 6.0):
        x = x.contiguous()
        y = torch.empty_like(x)
        n = x.numel()
        stream_ptr = torch.cuda.current_stream().cuda_stream

        lib.clipped_relu_forward_launcher(x.data_ptr(), y.data_ptr(),
                                          c_float(cap), n, stream_ptr)
        ctx.save_for_backward(x)
        ctx.cap = cap
        return y

    @staticmethod
    def backward(ctx, gy):
        (x,) = ctx.saved_tensors
        gy = gy.contiguous()
        gx = torch.empty_like(x)
        n = x.numel()
        stream_ptr = torch.cuda.current_stream().cuda_stream

        lib.clipped_relu_backward_launcher(x.data_ptr(), gy.data_ptr(), gx.data_ptr(),
                                           c_float(ctx.cap), n, stream_ptr)
        return gx, None          # None for cap (not learnable)

# convenience wrapper
def clipped_relu(x, cap=6.0):
    return ClippedReLU.apply(x, float(cap))


In [ ]:
# tensor
x = torch.randn(2_000_000, device='cuda')

# correctness
y_custom = clipped_relu(x, cap=6.0)
y_ref    = torch.clamp(x, 0.0, 6.0)
print("match:", torch.allclose(y_custom, y_ref, atol=1e-6))

# timing
torch.cuda.synchronize()
for _ in range(5): clipped_relu(x); torch.clamp(x, 0.0, 6.0)  # warm‑up

start = torch.cuda.Event(True); end = torch.cuda.Event(True)

start.record(); clipped_relu(x); end.record(); torch.cuda.synchronize()
print("custom CUDA:  %.3f ms" % start.elapsed_time(end))

start.record(); torch.clamp(x, 0.0, 6.0); end.record(); torch.cuda.synchronize()
print("torch.clamp: %.3f ms" % start.elapsed_time(end))


match: True
custom CUDA:  0.342 ms
torch.clamp: 0.215 ms
